In [1]:
from suffix_trees import STree
import suffix_tree
import pandas as pd

In [2]:
class STreeEx():
    def __init__(self, sTree):
        # 傳入原套件後綴樹
        self.sTree = sTree

    def lrs(self):
        # 最深非葉節點
        deepestNode = self._find_lrs(self.sTree.root)
        start = deepestNode.idx
        end = deepestNode.idx + deepestNode.depth
        return self.sTree.word[start:end]

    def _find_lrs(self, node):
        nodes = [self._find_lrs(n)
            for (n,_) in node.transition_links
            # 排除葉節點
            if n.transition_links != []]

        if nodes == []:
            return node
        
        deepestNode = max(nodes, key=lambda n: n.depth)
        return deepestNode

# Input File

In [3]:
input_file_path = "./input/TableA.txt"

In [4]:
content = []
recb_start = 0
recb_end = 0
tag = []
ids = []
classes = []
pathid = []
parentid = []
tecid = []
cecid = []
encoding = []
col = []
others = []
with open(input_file_path) as file:
    line = file.readline()
    while(line):
        line = file.readline()
        tmp = line.split('\t')
        if tmp[0] == "Content":
            content = tmp[1:]
        elif tmp[0] == "RecB":
            tmp = tmp[1:]
            for index in range(len(tmp)):
                if tmp[index] == "SDR":
                    recb_start = index
                elif tmp[index] == "EDR":
                    recb_end = index
        elif tmp[0] == "Tag":
            tag = tmp[1:]
        elif tmp[0] == "ID":
            ids = tmp[1:]
        elif tmp[0] == "Class":
            classes = tmp[1:]
        elif tmp[0] == "PathId":
            pathid = tmp[1:]
        elif tmp[0] == "ParentId":
            parentid = tmp[1:]
        elif tmp[0] == "TECId":
            tecid = tmp[1:]
        elif tmp[0] == "CECId":
            cecid = tmp[1:]
        elif tmp[0] == "Encoding":
            encoding = tmp[1:]
        elif tmp[0] == "Col Type":
            col = tmp[1:]
        else:
            others.append(tmp[1:])
others = others[0:-4]

# Translate node in to ascii code for suffix tree

Change `encode` to use different attribute to encode leafnodes.
- 0: **Pathid** + **Parentid** + **TECid** + **CECid** + **Encoding** + **Column Type**
- 1: **Id** + **Class** + **Pathid** + **CECid**
- 2: **Pathid** + **Parentid** + **CECid**
<br><br>
Leafnodes will have same encoding if they have same attribute.

In [16]:
encode = 2
whole_string = ""
test = ""
node_dict = {} # code -> node num
code_dict = {} # code -> content
index_dict = {} # code -> first index
index = 0
for node in range(len(pathid)):
    if encode == 0:
        code = pathid[node] + "," + parentid[node] + "," + tecid[node] + "," + cecid[node] + "," + encoding[node] + "," + col[node]
    elif encode == 1:
        code = ids[node] + "," + classes[node] + "," + pathid[node] + "," + cecid[node]
    elif encode == 2:
        code = pathid[node] + "," + parentid[node] + "," + cecid[node]
    #if 'O' in col[node]: continue
    if code not in node_dict.keys():
        node_dict[code] = index
        code_dict[code] = others[0][node]
        index_dict[code] = node
        whole_string += chr(index)
        index += 1
        if index == 45 or index == 32:
            index+=1
    else:
        whole_string += chr(node_dict[code])
print("Example: ")
if encode == 0:
    print(pathid[0] + "," + parentid[0] + "," + tecid[0] + "," + cecid[0] + "," + encoding[0] + "," + col[0])
elif encode == 1:
    print(ids[0] + "," + classes[0] + "," + pathid[0] + "," + cecid[0])
if encode == 2:
    print(pathid[0] + "," + parentid[0] + "," + cecid[0])
inv_node_dict = {v: k for k, v in node_dict.items()} # node num -> code
inv_index_dict = {v: k for k, v in index_dict.items()} # index num -> code
print("\nConvert to Unicode String:\n", whole_string)

Example: 
0,"1:[1, 2]", 

Convert to Unicode String:
  	
!"#$%&'()*!"#$%&'()*+,./.0.1.2.3.4.5.6.7.8.9.:.;.<.=.>.?.@.A.B.C.D.E.F.G.H.I.JJKLLMKNKOPJJKLLQNKOKJJKLLMKNKOKJJKLLMKNKOPJJKLLMQNKOKJJKLLMKNKOKJJKLLMKNKOKJJKLLMKNKOKJJKLLKNKOPJJKLLMKNKOPJJKLLMQNKOKJJKLLKNKOKRSTUVWXYZ[\]^_`abcdefghijklmnopqrstu


In [17]:
def find_all_indexes(input_str, search_str):
    l1 = []
    length = len(input_str)
    index = 0
    while index < length:
        i = input_str.find(search_str, index)
        if i == -1:
            return l1
        l1.append(i)
        index = i + 1
    return l1

# Longest repeat pattern

In [18]:
st = STree.STree(whole_string)
st = STreeEx(st)
lrs = str(st.lrs())
print("Length: ", len(lrs))
print(lrs)
for i in lrs:
    print("Content: ", code_dict[inv_node_dict[ord(i)]])

Length:  31
NKOKJJKLLMKNKOKJJKLLMKNKOKJJKLL
Content:  ( :: 145 :: 
Content:  Gathering :: 140 :: 
Content:  ) :: 147 :: 
Content:  Gathering :: 140 :: 
Content:  https://t.kfs.io/upload_images/105724/_____2019-11-26___4.50.59_medium.png :: 138 :: 
Content:  https://t.kfs.io/upload_images/105724/_____2019-11-26___4.50.59_medium.png :: 138 :: 
Content:  Gathering :: 140 :: 
Content:  KKBOX Hackathon Online 2020 作品票選 :: 141 :: 
Content:  KKBOX Hackathon Online 2020 作品票選 :: 141 :: 
Content:  113 Attendees :: 143 :: 
Content:  Gathering :: 140 :: 
Content:  ( :: 145 :: 
Content:  Gathering :: 140 :: 
Content:  ) :: 147 :: 
Content:  Gathering :: 140 :: 
Content:  https://t.kfs.io/upload_images/105724/_____2019-11-26___4.50.59_medium.png :: 138 :: 
Content:  https://t.kfs.io/upload_images/105724/_____2019-11-26___4.50.59_medium.png :: 138 :: 
Content:  Gathering :: 140 :: 
Content:  KKBOX Hackathon Online 2020 作品票選 :: 141 :: 
Content:  KKBOX Hackathon Online 2020 作品票選 :: 141 :: 
Content:  11

# Show maximal repeats

Ignore maximal repeats if length equal to 1.

In [19]:
tree = suffix_tree.Tree({'A': whole_string})
print("Total Maximal repeats: ", len(tree.maximal_repeats()), '\n')
group = {}
top_repeats = (0, '')
for C, path in tree.maximal_repeats():
    if len(path) == 1: continue
    count = 0
    print("Length: ", len(path), "Pattern: {", path, "}")
    for id_, path2 in tree.find_all (path):
        count += 1
    print("Count: ", count)
    if count > top_repeats[0]:
        top_repeats = (count, str(path).replace(' ', ''))
    for i in path:
        print("Content: ", code_dict[inv_node_dict[ord(i)]])
    if index_dict[inv_node_dict[ord(path[0])]] not in group.keys():
        group[index_dict[inv_node_dict[ord(path[0])]]] = []
        group[index_dict[inv_node_dict[ord(path[0])]]].append(path)
    else:
        group[index_dict[inv_node_dict[ord(path[0])]]].append(path)
    print("\n", "="*50, "\n")

Total Maximal repeats:  26 

Length:  26 Pattern: {       !  "  #  $  %  &  '  (  )  *  }
Count:  2
Content:  Lowest price :: 29 :: 
Content:  # :: 30 :: 
Content:  All price :: 31 :: 
Content:  # :: 30 :: 
Content:  Free :: 33 :: 
Content:  # :: 30 :: 
Content:  TWD 1-1000 :: 35 :: 
Content:  # :: 30 :: 
Content:  TWD 1000-2000 :: 37 :: 
Content:  # :: 30 :: 
Content:  TWD 2000-3000 :: 39 :: 
Content:  # :: 30 :: 
Content:  Upon TWD 3000 :: 41 :: 
Content:  # :: 30 :: 
Content:  Time :: 43 :: 
Content:  # :: 30 :: 
Content:  All time :: 45 :: 
Content:  # :: 30 :: 
Content:  Today :: 47 :: 
Content:  # :: 30 :: 
Content:  In a week :: 49 :: 
Content:  # :: 30 :: 
Content:  In a month :: 51 :: 
Content:  # :: 30 :: 
Content:  In two months :: 53 :: 
Content:  # :: 30 :: 


Length:  5 Pattern: { J J K L L }
Count:  12
Content:  https://t.kfs.io/upload_images/105724/_____2019-11-26___4.50.59_medium.png :: 138 :: 
Content:  https://t.kfs.io/upload_images/105724/_____2019-1

In [20]:
top_repeats

(12, 'JJKLL')

# Whole String split by top repeats

In [21]:
max_len = 0
seqs = []
pos = find_all_indexes(whole_string, top_repeats[1])
for idx in range(len(pos)):
    if idx+1 != len(pos):
        seqs.append(whole_string[pos[idx]:pos[idx+1]])
        if len(whole_string[pos[idx]:pos[idx+1]]) > max_len:
            max_len = len(whole_string[pos[idx]:pos[idx+1]])
    else:
        seqs.append(whole_string[pos[idx]:pos[idx]+max_len+4])
print(str(seqs))

['JJKLLMKNKOP', 'JJKLLQNKOK', 'JJKLLMKNKOK', 'JJKLLMKNKOP', 'JJKLLMQNKOK', 'JJKLLMKNKOK', 'JJKLLMKNKOK', 'JJKLLMKNKOK', 'JJKLLKNKOP', 'JJKLLMKNKOP', 'JJKLLMQNKOK', 'JJKLLKNKOKR\x0cS\x0cT']


# MSA

In [43]:
import cstar
print("First round MSA\n", "="*100)
scores = [1, -1, -3] # matchScore, mismatchScore, gapScore
msa = cstar.CenterStar(scores, seqs[:-1]).msa()
trans_dict = {}
last_c = '-'
end_idx = 0

for i in range(len(msa)):
    if msa[i][-1] != '-' and last_c not in trans_dict.keys():
        last_c = msa[i][-1]
        trans_dict[last_c] = msa[i].replace('-', '')
    if msa[i].replace('-', '') not in trans_dict.keys():
        trans_dict[msa[i].replace('-', '')] = msa[i]
    else: pass
for i in seqs[:-1]:
    print(i, "\t-> ", trans_dict[i])

print('='*100, "\nSecond round MSA\n")
msa_2 = cstar.CenterStar(scores, msa+[seqs[-1]]).msa()
trans_dict_2 = {}

for i in range(len(msa_2)):
    if msa_2[i].replace('-', '') not in trans_dict_2.keys():
        trans_dict_2[msa_2[i].replace('-', '')] = msa_2[i]
    else: pass
for idx in range(len(trans_dict_2[trans_dict[last_c]])):
    if trans_dict_2[trans_dict[last_c]][idx] == last_c:
        end_idx = idx
for i in seqs:
    trans_dict_2[i] = trans_dict_2[i][:end_idx+1]
    print(i, "\t-> ", trans_dict_2[i])

with open('./Set.txt', 'w', encoding='utf-8') as file:
    for page in range(len(others)):
        output_dict = {} # Record which pattern is used
        for s in range(len(seqs)):
            file.write(str(page) + "-0" + "-" + str(s) + "\t")
            tmp = find_all_indexes(whole_string, seqs[s])
            if seqs[s] not in output_dict.keys():
                output_dict[seqs[s]] = 0
            else:
                output_dict[seqs[s]] += 1
            #print("start:", tmp[output_dict[seqs[s]]], others[page][tmp[output_dict[seqs[s]]]])
            idx = 0
            for c in range(len(trans_dict_2[seqs[s]])):
                if trans_dict_2[seqs[s]][c] == '-':
                    file.write('\t')
                else:
                    file.write(others[page][tmp[output_dict[seqs[s]]]+idx] + "\t")
                    idx += 1
            file.write('\n')

First round MSA
JJKLLMKNKOP 	->  JJKLLMKNKOP
JJKLLQNKOK 	->  JJKLL-QNKOK
JJKLLMKNKOK 	->  JJKLLMKNKOK
JJKLLMKNKOP 	->  JJKLLMKNKOP
JJKLLMQNKOK 	->  JJKLLMQNKOK
JJKLLMKNKOK 	->  JJKLLMKNKOK
JJKLLMKNKOK 	->  JJKLLMKNKOK
JJKLLMKNKOK 	->  JJKLLMKNKOK
JJKLLKNKOP 	->  JJKLL-KNKOP
JJKLLMKNKOP 	->  JJKLLMKNKOP
JJKLLMQNKOK 	->  JJKLLMQNKOK
Second round MSA

JJKLLMKNKOP 	->  JJKLLMKNKOP
JJKLLQNKOK 	->  JJKLL-QNKOK
JJKLLMKNKOK 	->  JJKLLMKNKOK
JJKLLMKNKOP 	->  JJKLLMKNKOP
JJKLLMQNKOK 	->  JJKLLMQNKOK
JJKLLMKNKOK 	->  JJKLLMKNKOK
JJKLLMKNKOK 	->  JJKLLMKNKOK
JJKLLMKNKOK 	->  JJKLLMKNKOK
JJKLLKNKOP 	->  JJKLL-KNKOP
JJKLLMKNKOP 	->  JJKLLMKNKOP
JJKLLMQNKOK 	->  JJKLLMQNKOK
JJKLLKNKOKRST 	->  JJKLL-KNKOK
